# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### All' Antico Vinaio - Toscana

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187895-d1102253-Reviews-All_Antico_Vinaio-Florence_Tuscany.html

Home --> Ristoranti --> Firenze --> Best_ranking--> SENZ_ALTRO_Bistrot

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187895-d1102253-Reviews-All_Antico_Vinaio-Florence_Tuscany.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187895-d1102253-Reviews-or10-All_Antico_Vinaio-Florence_Tuscany.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187895-d1102253-Reviews-or'
URL_p2 = '0-All_Antico_Vinaio-Florence_Tuscany.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in range(1,total_pages):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
total_pages:  2031
Nomi:  20305
Commenti:  20305
Voti:  20305
Scritto da:  20305


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [0]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 20305 
 Recensioni analizzabili: 12826 
 Recensioni scartate: 7479 



['RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Panini squisiti e con prodotti locali, ragazzi veloci e simpatici! Ci sono 4 locali e bisogna attendere ma ne vale la pena',
 'File kilometriche non devono scoraggiare.\nCibo top\nPanini top\nVino top\nNe vale davvero la pena\nPer un panino succulento',
 'RECENSIONE_TROPPO_LUNGA',
 'Locale storico, ci vo da tutta la vita!!!!!!!\nPiccolo onesto e sincero, ottimi prodotti, mi piace la schiacciata,\nservizio veloce anche se non riescono a smaltire la coda (unico difetto)\nASSOLUTAMENTE DA PROVARE per chi viene a Firenze',
 "Ottima schiacciata con salumi, formaggi e sott'oli locali. Pane e condimento veraci come il locale. Sono stato fortunato a far poca fila. Non escludo di ritornare perché ho notato dopo il bicchiere di vino self - service ;)",
 'RECENSIONE_TROPPO_LUNGA',
 'Fila chilometrica smaltita in pochi minuti, schiacciate buonissime, buonumore con le battute del personale, vino a volontà. Prezzo concorrenziale. Che dire di 

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['brambybaby', 'Agata_Turistapercaso', 'Carla l', 'TrailBlazer822585', 'Prinvio', 'Massimo B', 'marcoaU295BM', 'silviaivlis', 'davidarx', 'Ilataranto', 'SilviaLuc17', 'monicar520', 'Massssssss', 'Gianfrancocuoco', 'cdavide79', 'giulia1500', 'Simonemau', 'morenomandelli', 'maisolaatavola', 'saimonsays', '419alessandrab', 'valentinab3591', 'cri1950', 'Navigate652881', 'Floriano_Ferrante', 'francescafW2077JQ', 'Stefano V', 'julsina66', 'Stefaniaguarguagli', '586lorenzod', 'elisa d', 'gherardobaudo', 'cardo90', '23eceg23', 'Giulia P', 'robertoz642', 'Lorenzo B', '_lucacortesi99', '463giulian', 'Lucrezia T', 'mariopW8725TQ', 'andreabertaz', 'Umberto V', 'jishunz2018', 'Giuseppe D', 'alicep31', 'Diana L', 'cosim64', '582emanueles', 'tiaf601', '631gabriellan', 'MartinaS560', 'Deborah G', 'P7435ROdiegof', 'ila_ce4', 'Angelo M', 'Salvo R', 'L9300PEmatteov', 'P550MZmatteop', 'claudguidi', 'jacopod691', 'lillybb2018', 'silvia r', 'evuzy', '911khrystynar', 'Ylenia T', 'Franco R', 'Giorgio A', 'sab

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

    100% |████████████████████████████████| 389kB 27.2MB/s 


In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'male', 'unknown', 'male', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unkno

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'M', 'M', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN',

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  20305
Commenti:  20305
Voti:  20305
Mobile:  20305


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  3084


[['Carla',
  'F',
  'Panini squisiti e con prodotti locali, ragazzi veloci e simpatici! Ci sono 4 locali e bisogna attendere ma ne vale la pena',
  '5',
  'Mobile'],
 ['Massimo',
  'M',
  'Locale storico, ci vo da tutta la vita!!!!!!!\nPiccolo onesto e sincero, ottimi prodotti, mi piace la schiacciata,\nservizio veloce anche se non riescono a smaltire la coda (unico difetto)\nASSOLUTAMENTE DA PROVARE per chi viene a Firenze',
  '5',
  'PC'],
 ['Stefano',
  'M',
  "No. Puoi passare da Firenze senza vedere il duomo, piazzale michelangelo, gli uffizi, i georgofili ma principalmente senza passare ad assaggiare la schiacciata dell'antico vinaio.",
  '5',
  'Mobile'],
 ['Giulia',
  'F',
  "La presentazione è pazzesca, la coda chilometrica sembra promette meraviglie, ma il risultato è un po' deludente. La schiacciata era complessivamente gelida, salatissima e la crema al tartufo indigesta. forse le aspettative altissime dovute alla pubblicizzazione social non hanno aiutato... Gran dispiacere!

### Creiamo la tabella

In [18]:
!pip install xlsxwriter
import pandas as pd

    100% |████████████████████████████████| 143kB 8.3MB/s 


In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_13.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [20]:
partial_dataframe = pd.read_excel("RISTORANTE_13.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,F,Carla,"Panini squisiti e con prodotti locali, ragazzi...",Mobile,5
1,M,Massimo,"Locale storico, ci vo da tutta la vita!!!!!!!\...",PC,5
2,M,Stefano,No. Puoi passare da Firenze senza vedere il du...,Mobile,5
3,F,Giulia,"La presentazione è pazzesca, la coda chilometr...",Mobile,2
4,F,Lucrezia,"Una buonissima schiacciata, con ingredienti fr...",PC,5
5,M,Umberto,Sempre file all’Antico Vinaio. Però sono veloc...,PC,5
6,M,Giuseppe,Focacceria numero 1 nel cuore di Firenze a poc...,Mobile,5
7,M,Angelo,"Street food consigliato,coda infinita,ma ne va...",Mobile,4
8,F,Giulia,È stata la prima tappa della nostra visita a F...,Mobile,5
9,M,Luca,Tre mesi che aspettavamo di venire e non ne si...,Mobile,5
